# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import processingmm

 Could not import win32api and/or win32con
 [wrn] tensorboard not available - training not possible
 [wrn] tensorboard not available - training not possible


In [6]:
# set the parameters to run the script
directories = [r'/media/elea/ssd/NPP/test']
calib_directory = r'/media/elea/ssd/NPP/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'no'

# do not specify unless you want to use a custom path for the PDDN models
PDDN_models_path = None

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550, 600]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN (with the data on a SSD drive)
# 'no_viz': 0.39s
# 'default', save_pdf_figs False: 1.65s
# 'default', save_pdf_figs True: 2.67s
# 'full', save_pdf_figs False: 3.24s
# 'full', save_pdf_figs True: 5.94s
processing_mode = 'no_viz'
save_pdf_figs = False

# define if the wavelenghts should be aligned before processing - and used for the computation
align_wls = True

parameters = processingmm.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, PDDN_models_path = PDDN_models_path, 
                                processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs, align_wls = align_wls)

In [7]:
%%time
times = processingmm.batch_process_master(parameters)
times

processing without PDDN...
Aligning wavelength: 600 nm...


100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 15087.42it/s]


Aligning wavelengths done.



  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing: /media/elea/ssd/NPP/test/2022-07-06_T_AUTOPSY-BF_FR_S1_1


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.76it/s]

processing without PDDN done.
CPU times: user 3.92 s, sys: 241 ms, total: 4.16 s
Wall time: 574 ms


{'azimuth_curation': 0.007475852966308594,
 'data_loading': 0.015058517456054688,
 'MM_processing': 0.4295804500579834,
 'azimuth_std_processing': 0.05463552474975586,
 'full_processing': 0.5069880485534668,
 'save_npz': 0.05886983871459961,
 'viz': 4.5299530029296875e-05,
 'total': 0.5701749324798584}

In [6]:
from processingmm.addons import predict
predict.batch_prediction(parameters, no_labels = False, MM = False, model_name = None)

INFO: Using device cuda
INFO: Model loaded from intensities_1.pt
Steps 1 test:   0%|          | 0/1 [00:00<?, ?img/s]

Processing with labels...

Running predictions on sample 550/TU/2023-01-04_T_HORAO-77-BF_FR_S3_1


Steps 1 test:   0%|          | 0/1 [00:00<?, ?img/s]


Running predictions on sample 550/TU/2022-09-14_T_HORAO-39-AF_FR_S1_1


Steps 1 test:   0%|          | 0/1 [00:00<?, ?img/s]

...

Steps 1 test:   0%|          | 0/1 [00:00<?, ?img/s]

Sample ID 2022-07-06_T_AUTOPSY-BF_FR_S8_1 not found in metadata.

Running predictions on sample 550/TU/2023-10-19_T_HORAO-107-AF_FR_S3_1


Steps 1 test:   0%|          | 0/1 [00:00<?, ?img/s]


Running predictions on sample 550/TU/2023-01-04_T_HORAO-77-BF_FR_S1_1


Steps 1 test: 100%|██████████| 1/1 [00:00<00:00,  1.09img/s]


In [129]:
%%time
from processingmm.addons import visualization_lines

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
run_all = False
times = visualization_lines.batch_visualization(parameters, run_all)
times

CPU times: user 280 μs, sys: 1.02 ms, total: 1.3 ms
Wall time: 1.08 ms


0.0005217790603637695

In [131]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
directories = ['/media/elea/ssd/NPP_dev/test']

PDDN_mode = 'both'
run_all = False
align_wavelengths.align_wavelengths(directories, PDDN_mode, run_all, [600])

Aligning wavelength: 600 nm...


100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 11554.56it/s]

Aligning wavelengths done.

